# 0. Load data

In [1]:
import pandas as pd
from cleaning_scripts.load_data_to_clean2 import load_data_to_clean
from cleaning_scripts.string_operations import create_artist_name_col, create_artwork_title
from cleaning_scripts.acquisition_operations import *
from cleaning_scripts.production_operations import *
from cleaning_scripts.outliers_operations import find_outliers
raw_data, dataset_names = load_data_to_clean()
transformed_data = []
museums_opening_date = [1870, 1990, 1897, 1977, 1929, 1931, 1937, 1998, 1896, 1887, 1895]

Loaded met with shape: (476318, 54)
Loaded reina_sofia with shape: (10018, 16)
Loaded tate with shape: (69201, 31)
Loaded pompidou with shape: (115217, 23)
Loaded moma with shape: (157474, 30)
Loaded whitney with shape: (26497, 21)
Loaded national_gallery with shape: (141438, 55)
Loaded kiasma with shape: (8329, 70)
Loaded smk with shape: (175525, 17)
Loaded ateneum with shape: (27994, 88)
Loaded queensland with shape: (20250, 22)


# 1. use functions for cleaning fields.
artist name, artwork title

In [2]:
#create a new column with Artist name
for idx, df in zip(dataset_names, raw_data):
    print('-----------------------------------')
    print(idx)
    create_artist_name_col(df)
    transformed_data.append(df)

-----------------------------------
met
Artist name found in column: Artist Display Name
-----------------------------------
reina_sofia
Artist name found in column: author
-----------------------------------
tate
Artist name found in column: artist
-----------------------------------
pompidou
Artist name found in column: artist_name
-----------------------------------
moma
Artist name found in column: Artist
-----------------------------------
whitney
Artist name found in column: artists
-----------------------------------
national_gallery
Artist name found in column: forwarddisplayname
-----------------------------------
kiasma
Artist name found in column: artist_name
-----------------------------------
smk
Artist name found in column: artist_0
-----------------------------------
ateneum
Artist name found in column: artist_name
-----------------------------------
queensland
Artist name found in column: Artist


In [3]:
#create a new column with Artwork title
for idx, df in zip(dataset_names, transformed_data):
    print(idx)
    create_artwork_title(df)

met
Artwork title found in column: Title
reina_sofia
Artwork title found in column: name
tate
Artwork title found in column: name
pompidou
Artwork title found in column: object_title
moma
Artwork title found in column: Title
whitney
Artwork title found in column: title
national_gallery
Artwork title found in column: title
kiasma
Artwork title found in column: title_fi
smk
Artwork title found in column: titles_0_title
ateneum
Artwork title found in column: title_fi
queensland
Artwork title found in column: Title


# 2. aquisition year

In [4]:
for idx, df in zip(dataset_names, transformed_data):
    print(idx)
    clean_acquisition_year(df)

met
Data found in column: AccessionYear
Number of cleaned variables: 471928 out of 476318 (99.1%)
----------------------------------
reina_sofia
Data found in column: year_adquisition
Number of cleaned variables: 9951 out of 10018 (99.3%)
----------------------------------
tate
Data found in column: acquisitionYear
Number of cleaned variables: 69156 out of 69201 (99.9%)
----------------------------------
pompidou
Data found in column: acquisition_date
Number of cleaned variables: 100340 out of 115217 (87.1%)
----------------------------------
moma
Data found in column: DateAcquired
Number of cleaned variables: 150772 out of 157474 (95.7%)
----------------------------------
whitney
Data found in column: credit_line
Number of cleaned variables: 115 out of 26497 (0.4%)
----------------------------------
national_gallery
Data found in column: accessionnum
Number of cleaned variables: 141430 out of 141438 (100.0%)
----------------------------------
kiasma
Data found in column: inventoryNumb

In [5]:
diagnostic_df = analyze_acquisition_columns(transformed_data[9])

In [6]:
improve_acquisition_pompidou(transformed_data[3])
improve_acquisition_whitney(transformed_data[5])
improve_acquisition_nga(transformed_data[6])
improve_acquisition_kiasma(transformed_data[7])
improve_acquisition_ateneum(transformed_data[9])


Fixed 1263 out of 1933 problematic rows
Pompidou - Number of cleaned variables: 101603 out of 115217 (88.2%)
---------------------------------------
Fixed 26378 out of 26378 problematic rows
Whitney - Number of cleaned variables: 26493 out of 26497 (100.0%)
---------------------------------------
Fixed 7 out of 7 problematic rows
Natonal Gallery - Number of cleaned variables: 141437 out of 141438 (100.0%)
---------------------------------------
Fixed 9 out of 430 problematic rows
Kiasma - Number of cleaned variables: 7906 out of 8329 (94.9%)
---------------------------------------
Fixed 11839 out of 22867 problematic rows
Ateneum - Number of cleaned variables: 16965 out of 27994 (60.6%)
---------------------------------------


,Unnamed: 0,objectId,responsibleOrganisation,dimensions_0,dimensions_1,dimensionUnit,acquisitionYear,title_en,title_fi,category_categoryId,...,people_8_deathYear,people_9_id,people_9_firstName,people_9_familyName,medium,artist_name,Artist,Title,Year_acquisition,source_column
0,1,438766,Kansallisgalleria / Ateneumin taidemuseo,23.5,23.5,cm,NaN,NaN,"Malliharjoitelma, nainen, jolla on mustat suka...",artwork,...,NaN,NaN,NaN,NaN,black chalk mustaliitu svartkrita mustaliitu,Alfred William Finch,Alfred William Finch,"Malliharjoitelma, nainen, jolla on mustat suka...",<NA>,inventoryNumber
1,2,446767,Kansallisgalleria / Ateneumin taidemuseo,29.0,42.0,cm,NaN,NaN,Hevonen,artwork,...,NaN,NaN,NaN,NaN,pencil lyijykynä blyertspenna lyijykynä,Alfred William Finch,Alfred William Finch,Hevonen,<NA>,inventoryNumber
2,6,462031,Kansallisgalleria / Ateneumin taidemuseo,22.0,17.6,cm,NaN,"Zvicken the Sleeping Dog, study","Zvicken, nukkuva koira, harjoitelma",artwork,...,NaN,NaN,NaN,NaN,mustekynä,Akseli Gallen-Kallela,Akseli Gallen-Kallela,"Zvicken, nukkuva koira, harjoitelma",<NA>,inventoryNumber
3,8,444040,Kansallisgalleria / Ateneumin taidemuseo,180.5,100.0,cm,NaN,NaN,"Juominki, luonnos Grand Cafén seinämaalaukseen...",artwork,...,NaN,NaN,NaN,NaN,kangas oil öljy olja oil on canvas öljy kankaa...,Per Krohg,Per Krohg,"Juominki, luonnos Grand Cafén seinämaalaukseen...",<NA>,inventoryNumber
4,11,610341,Kansallisgalleria / Ateneumin taidemuseo,120.0,150.0,cm,2003.0,Street Scene,Katukuva,artwork,...,NaN,NaN,NaN,NaN,kangas oil öljy olja oil on canvas öljy kankaa...,Ulla Rantanen,Ulla Rantanen,Katukuva,2003,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27989,56771,539754,Kansallisgalleria / Ateneumin taidemuseo,19.6,14.5,cm,NaN,NaN,Juutalaismorsian,artwork,...,NaN,NaN,NaN,NaN,etching viivasyövytys etsning,William Unger,William Unger,Juutalaismorsian,<NA>,inventoryNumber
27990,56879,1784430,Kansallisgalleria / Ateneumin taidemuseo,46.5,64.0,cm,NaN,Codfish,Turska,artwork,...,NaN,NaN,NaN,NaN,kangas oil öljy olja oil on canvas öljy kankaa...,Åke Hellman,Åke Hellman,Turska,2019,inventoryNumber
27991,57398,541358,Kansallisgalleria / Ateneumin taidemuseo,14.6,20.6,cm,NaN,NaN,Metsästäjiä lähteellä,artwork,...,NaN,NaN,NaN,NaN,etching viivasyövytys etsning,William Unger,William Unger,Metsästäjiä lähteellä,<NA>,inventoryNumber
27992,58062,590062,Kansallisgalleria / Ateneumin taidemuseo,23.8,33.2,cm,NaN,NaN,Saaristomaisema,artwork,...,NaN,NaN,NaN,NaN,etching viivasyövytys etsning,Frans Maexmontan,Frans Maexmontan,Saaristomaisema,1923,inventoryNumber


In [7]:

for idx, date, df in zip(dataset_names, museums_opening_date, transformed_data):
    print('------------------------------')
    print(idx)
    print(f'museum opened in {date}')
    find_outliers(df, 'Year_acquisition')

------------------------------
met
museum opened in 1870

Year statistics:
Earliest year: 1870
Latest year: 2021
------------------------------
reina_sofia
museum opened in 1990

Year statistics:
Earliest year: 1888
Latest year: 2021

Potential outliers detected (2 values):
Year_acquisition
1888    1
1974    1
Name: count, dtype: int64
----------------------------------
------------------------------
tate
museum opened in 1897

Year statistics:
Earliest year: 1823
Latest year: 2013
------------------------------
pompidou
museum opened in 1977

Year statistics:
Earliest year: 1897
Latest year: 2021

Potential outliers detected (4980 values):
Year_acquisition
1957    2947
1937     170
1927     128
1939     123
1931     120
Name: count, dtype: int64
----------------------------------
------------------------------
moma
museum opened in 1929

Year statistics:
Earliest year: 1929
Latest year: 2024
------------------------------
whitney
museum opened in 1931

Year statistics:
Earliest year: 

# 4. creation year 

In [8]:
for idx, df in zip(dataset_names, transformed_data):
    print(idx)
    artwork_creation_date(df)

met
Data found in column: Object Date
Number of cleaned variables: 289749 out of 476318 (60.8%)
----------------------------------
reina_sofia
Data found in column: year_production
Number of cleaned variables: 10010 out of 10018 (99.9%)
----------------------------------
tate
Data found in column: year
Number of cleaned variables: 63805 out of 69201 (92.2%)
----------------------------------
pompidou
Data found in column: object_date
Number of cleaned variables: 110546 out of 115217 (95.9%)
----------------------------------
moma
Data found in column: Date
Number of cleaned variables: 151988 out of 157474 (96.5%)
----------------------------------
whitney
Data found in column: display_date
Number of cleaned variables: 25924 out of 26497 (97.8%)
----------------------------------
national_gallery
Data found in column: endyear_x
Number of cleaned variables: 140925 out of 141438 (99.6%)
----------------------------------
kiasma
Data found in column: yearFrom
Number of cleaned variables: 7

In [9]:
diagnostic_df = analyze_production_columns(transformed_data[6])

In [10]:
improve_production_met(transformed_data[0])
improve_production_moma(transformed_data[4])
improve_production_whitney(transformed_data[5])
improve_production_nga(transformed_data[6])
improve_production_queensland(transformed_data[10])

Fixed 116120 out of 172525 problematic rows
MET - Number of cleaned variables: 405869 out of 476318 (85.2%)
---------------------------------------
Fixed 1026 out of 3473 problematic rows
MOMA - Number of cleaned variables: 153014 out of 157474 (97.2%)
---------------------------------------
Fixed 100 out of 569 problematic rows
whitney - Number of cleaned variables: 26024 out of 26497 (98.2%)
---------------------------------------
Fixed 0 out of 43 problematic rows
NGA - Number of cleaned variables: 140925 out of 141438 (99.6%)
---------------------------------------
Fixed 675 out of 1298 problematic rows
Queensland - Number of cleaned variables: 19057 out of 20250 (94.1%)
---------------------------------------


,Unnamed: 0,_id,irn,PlaceCreated,DateCreated,PhyMediumText,CreditLine,Person,Title,PhyMediaCategory,...,AcquiredDate,PhysicalCategory,CollectionOnlineAPI,Artist,Nationality,birth_date,death_date,Year_acquisition,Date_creation_year,source_column
0,0,1,1,NaN,1961,Watercolour on wove paper,Acquired before 1961,"GIBSON, Bessie\nQueensland Australia 1868–1961",Portrait (woman knitting),watercolour,...,1961,Painting,https://collection.qagoma.qld.gov.au/api/objec...,"GIBSON, Bessie",Queensland Australia,1868.0,1961.0,1961,1961,nan
1,1,2,2,NaN,1959,Enamel on composition board,Purchased 1963,"DICKERSON, Robert\nNew South Wales Australia 1...",Working in the fields,enamel,...,25/09/1963,Painting,https://collection.qagoma.qld.gov.au/api/objec...,"DICKERSON, Robert",New South Wales Australia,1924.0,2015.0,1963,1959,nan
2,2,3,4,NaN,1965,Synthetic polyester resin with talc and integr...,Gift of Mrs Lillian Bosch 1972,"JOMANTAS, Vincas\nLithuania/Australia VIC 1922...",Sacrifice II,Sculpture,...,28/08/1972,Sculpture,https://collection.qagoma.qld.gov.au/api/objec...,"JOMANTAS, Vincas",Lithuania/Australia VIC,1922.0,2001.0,1972,1965,nan
3,3,4,5,NaN,"1745, repr. 19th C.","Line and stipple engraving, etching on buff wo...",Gift of Mrs Jane E. Watson 1932,"HOGARTH, William (Printer)\nEngland United Kin...","Marriage a la Mode, plate II. The breakfast scene",intaglio,...,4/03/1932,Print,https://collection.qagoma.qld.gov.au/api/objec...,"HOGARTH, William (Printer)",France,1697.0,1764.0,1932,1745,nan
4,4,5,6,NaN,c.1926-33,Reproduction on paper,Gift of J.H. Wood 1940,"LOW, after David\nNew Zealand/Australia/Englan...",Aldous Huxley (reproduction),photo-mechanical,...,7/10/1940,Print,https://collection.qagoma.qld.gov.au/api/objec...,"LOW, after David",New Zealand/Australia/England NSW,1892.0,1963.0,1940,1926,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20245,20245,20246,43672,NaN,1985-86,Type C photograph on paper,Gift of an anonymous donor through the Queensl...,"HENSON, Bill - Creator",Untitled #73,NaN,...,30/11/2022,Photograph,https://collection.qagoma.qld.gov.au/api/objec...,"HENSON, Bill - Creator","HENSON, Bill - Creator",NaN,NaN,2022,1985,nan
20246,20246,20247,43676,NaN,2022,"Timber, poly-coated wire",The Kenneth and Yasuko Myer Collection of Cont...,"RYUI, Koji - Creator",Petit pois #2,NaN,...,15/11/2022,Sculpture,https://collection.qagoma.qld.gov.au/api/objec...,"RYUI, Koji - Creator","RYUI, Koji - Creator",NaN,NaN,2022,2022,nan
20247,20247,20248,43677,NaN,2022,"Timber, plaster, clay, oxides",The Kenneth and Yasuko Myer Collection of Cont...,"RYUI, Koji - Creator",Etymology,NaN,...,15/11/2022,Sculpture,https://collection.qagoma.qld.gov.au/api/objec...,"RYUI, Koji - Creator","RYUI, Koji - Creator",NaN,NaN,2022,2022,nan
20248,20248,20249,43678,NaN,2022,"Timber, found objects, cork, pins, plaster, pa...",The Kenneth and Yasuko Myer Collection of Cont...,"RYUI, Koji - Creator",Calico crown,NaN,...,15/11/2022,Sculpture,https://collection.qagoma.qld.gov.au/api/objec...,"RYUI, Koji - Creator","RYUI, Koji - Creator",NaN,NaN,2022,2022,nan


In [11]:
for idx, df in zip(dataset_names, transformed_data):
    print('------------------------------')
    print(idx)
    find_outliers(df, 'Date_creation_year')

------------------------------
met



Year statistics:
Earliest year: 1000
Latest year: 2021

Potential outliers detected (22591 values):
Date_creation_year
1390    2863
1295    2394
1300    2167
1199    1722
1399    1266
Name: count, dtype: int64
----------------------------------
------------------------------
reina_sofia

Year statistics:
Earliest year: 1843
Latest year: 2018

Potential outliers detected (17 values):
Date_creation_year
1867    3
1844    3
1845    3
1843    2
1870    2
Name: count, dtype: int64
----------------------------------
------------------------------
tate

Year statistics:
Earliest year: 1545
Latest year: 2012

Potential outliers detected (18 values):
Date_creation_year
1545    2
1573    2
1596    2
1576    1
1600    1
Name: count, dtype: int64
----------------------------------
------------------------------
pompidou

Year statistics:
Earliest year: 1000
Latest year: 2020

Potential outliers detected (54 values):
Date_creation_year
1850    22
1800    11
1859     4
1858     3
1810     2
Name: c

# 5. Artist birth

In [12]:
from cleaning_scripts.cleaning_dates import *
possible_columns = ['Artist Begin Date', 'author_born_year', 'yearOfBirth', 'artist_birth', 'BeginDate', 'birth_date', 
                        'beginyear', 'people_0_birthYear', 'production_0_creator_date_of_birth']

for idx, df in zip(dataset_names, transformed_data):
    print(idx)
    extract_date(df, possible_columns, 'Artist_birth_year')

met
Data found in column: Artist Begin Date
Number of cleaned variables: 236129 out of 476318 (49.6%)
----------------------------------
reina_sofia
Data found in column: author_born_year
Number of cleaned variables: 9763 out of 10018 (97.5%)
----------------------------------
tate
Data found in column: yearOfBirth
Number of cleaned variables: 68657 out of 69201 (99.2%)
----------------------------------
pompidou
Data found in column: artist_birth
Number of cleaned variables: 83543 out of 115217 (72.5%)
----------------------------------
moma
Data found in column: BeginDate
Number of cleaned variables: 148242 out of 157474 (94.1%)
----------------------------------
whitney
Data found in column: birth_date
Number of cleaned variables: 25930 out of 26497 (97.9%)
----------------------------------
national_gallery
Data found in column: beginyear
Number of cleaned variables: 135119 out of 141438 (95.5%)
----------------------------------
kiasma
Data found in column: people_0_birthYear
Numb

In [13]:
diagnostic_df = analyze_data_quality(transformed_data[10], possible_columns, 'Artist_birth_year')

Data taken from column birth_date


In [14]:
extract_date_from_other_column(transformed_data[1], 'Artist_birth_year', 'author_born_year', 'born-death-raw')

Fixed 108 out of 255 problematic rows
MET - Number of cleaned variables: 9871 out of 10018 (98.5%)
---------------------------------------


# 6. Artist death

In [15]:
possible_columns = ['Artist End Date', 'author_death_year', 'yearOfDeath', 'artist_death', 'EndDate', 'death_date', 'endyear', 
                        'people_0_deathYear', 'production_0_creator_date_of_death']
for idx, df in zip(dataset_names, transformed_data):
    print(idx)
    extract_date(df, possible_columns, 'Artist_death_year')

met
Data found in column: Artist End Date
Number of cleaned variables: 219364 out of 476318 (46.1%)
----------------------------------
reina_sofia
Data found in column: author_death_year
Number of cleaned variables: 6551 out of 10018 (65.4%)
----------------------------------
tate
Data found in column: yearOfDeath
Number of cleaned variables: 60427 out of 69201 (87.3%)
----------------------------------
pompidou
Data found in column: artist_death
Number of cleaned variables: 83461 out of 115217 (72.4%)
----------------------------------
moma
Data found in column: EndDate
Number of cleaned variables: 109139 out of 157474 (69.3%)
----------------------------------
whitney
Data found in column: death_date
Number of cleaned variables: 16451 out of 26497 (62.1%)
----------------------------------
national_gallery
Data found in column: endyear
Number of cleaned variables: 123658 out of 141438 (87.4%)
----------------------------------
kiasma
Data found in column: people_0_deathYear
Number of

In [16]:
diagnostic_df = analyze_data_quality(transformed_data[10], possible_columns, 'Artist_death_year')

Data taken from column death_date


In [17]:

extract_last_date_from_other_column(transformed_data[1], 'Artist_birth_year', 'author_born_year', 'born-death-raw')

Fixed 0 out of 147 problematic rows
MET - Number of cleaned variables: 9871 out of 10018 (98.5%)
---------------------------------------


# 7. artist nationality, artist gender, artwork medium, and acquisition source,

In [18]:
#I need to add more tags based on dataset values
from cleaning_scripts.medium_tags import load_medium_tags #returns medium_tags, medium_name
medium_tags, medium_name = load_medium_tags()
def classify_medium(df):
    df['Medium_classified'] = pd.NA
    
     # Iterate over each row in the DataFrame
    for idx, row in df.iterrows():
        medium = row['Medium_raw'] 
        
        # Skip if the 'Medium' value is not a string (e.g., NaN or other types)
        if not isinstance(medium, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(medium_tags):
            # Check if any keyword from the tag group exists in the 'Medium' column
            if any(tag.lower() in medium.lower() for tag in tag_group):
                # Assign the corresponding medium name (from medium_name list)
                df.at[idx, 'Medium_classified'] = medium_name[i]
                break  # Stop once we find a match
    
    return df

In [19]:
#i need to update the dictionary so all nationalities will be translated into countries
from cleaning_scripts.load_nationality_tags import load_nationality_tags
nationality_tags, country_name = load_nationality_tags()
def create_artist_nationality(df):
    possible_columns = ['Country','Artist Nationality', 'nationality_artist', 'artist_nationality', 'Nationality', 'nationality', 
                        'production_0_creator_nationality']
    df['Country_calculated'] = pd.NA
    df['nationality_raw'] = pd.NA

    # Check each possible column for nationality information
    for col in possible_columns:
        if col in df.columns:
            df['nationality_raw'] = df[col].str.lower().str.strip('the').str.replace('(', '').str.replace(')', '').str.split(' ').str[0].str.strip(',')  # Copy the nationality info into 'nationality_raw'
            break  # Exit loop once we find the first match 

    for idx, row in df.iterrows():
        nationality = row['nationality_raw'] 
        
        # Skip if the 'nationality' value is not a string (e.g., NaN or other types)
        if not isinstance(nationality, str):
            continue
        
        # Check each tag group (e.g., ['paper', 'watercolor'], ['oil'])
        for i, tag_group in enumerate(nationality_tags):
            # Check if any keyword from the tag group exists in the 'nationality' column
            if any(tag.lower() in nationality.lower() for tag in tag_group):
                # Assign the corresponding nationality name (from nationality_name list)
                df.at[idx, 'Country_calculated'] = country_name[i]
                break  # Stop once we find a match

    return df


use functions for raw datasets and dreate a new list with clean datasets

In [20]:
def create_artist_gender(df):
    possible_columns = ['Artist Gender', 'gender', 'artist_gender', 'Gender', 'production_0_creator_gender']
    # Define a gender dictionary to map common terms to 'female' or 'male'
    gender_dict = {'female': ["female", 'woman'], 'male': ['male', 'man']}
    
    # Initialize the 'Gender_raw' column with missing values
    df['Gender_raw'] = pd.NA
    
    # Check which column exists and assign to 'Gender_raw'
    for col in possible_columns:
        if col in df.columns:
            df['Gender_raw'] = df[col]
            break  # Exit loop once we find the first match
    
    # Clean the text: remove all non-alphanumeric characters, split by spaces, and use the first word
    df['Gender_classified'] = df['Gender_raw'].str.replace(r'[^a-zA-Z\s]', '', regex=True)  # Remove non-alphabetic characters
    df['Gender_classified'] = df['Gender_classified'].str.split().str[0].str.lower()  # Split by space and take the first word

    # Manually map "man" and "woman" to "male" and "female" directly
    df['Gender_classified'] = df['Gender_classified'].replace({'man': 'male', 'woman': 'female'})
    
    # Use a safe check to apply gender classification only on valid strings
    df['Gender_classified'] = df['Gender_classified'].apply(
        lambda x: 'female' if isinstance(x, str) and 'female' in x else ('male' if isinstance(x, str) and 'male' in x else x)
    )
    
    # Return the modified DataFrame
    return df


In [22]:
from cleaning_scripts.load_acquisition_tags import load_acquisition_tags
acqusition_tags, acquisition_methods = load_acquisition_tags()
def create_acquisition_method(df):
    possible_columns = ['Credit Line', 'creditLine', 'acquisition_type', 'CreditLine', 'credit_line', 'creditline', 'CreditLine']
     # Initialize the 'Gender_raw' column with missing values
    df['acquisition_raw'] = pd.NA
    
    # Check which column exists and assign to 'Gender_raw'
    for col in possible_columns:
        if col in df.columns:
            df['acquisition_raw'] = df[col].str.lower()
            break  # Exit loop once we find the first match

    for idx, row in df.iterrows():
        acquisition = row['acquisition_raw'] 
        
        # Skip if the 'acquisition' value is not a string (e.g., NaN or other types)
        if not isinstance(acquisition, str):
            continue
        
        # Check each tag group 
        for i, tag_group in enumerate(acqusition_tags):
            # Check if any keyword from the tag group exists in the 'acquisition' column
            if any(tag.lower() in acquisition.lower() for tag in tag_group):
                # Assign the corresponding acquisition name (from acquisition_name list)
                df.at[idx, 'Acquistion_classified'] = acquisition_methods[i]
                break  # Stop once we find a match
    return df
    

In [ ]:
transformed_data[0].columns

Index(['Object Number', 'Is Highlight', 'Is Timeline Work', 'Is Public Domain',
       'Object ID', 'Gallery Number', 'Department', 'AccessionYear',
       'Object Name', 'Title', 'Culture', 'Period', 'Dynasty', 'Reign',
       'Portfolio', 'Constituent ID', 'Artist Role', 'Artist Prefix',
       'Artist Display Name', 'Artist Display Bio', 'Artist Suffix',
       'Artist Alpha Sort', 'Artist Nationality', 'Artist Begin Date',
       'Artist End Date', 'Artist Gender', 'Artist ULAN URL',
       'Artist Wikidata URL', 'Object Date', 'Object Begin Date',
       'Object End Date', 'Medium', 'Dimensions', 'Credit Line',
       'Geography Type', 'City', 'State', 'County', 'Country', 'Region',
       'Subregion', 'Locale', 'Locus', 'Excavation', 'River', 'Classification',
       'Rights and Reproduction', 'Link Resource', 'Object Wikidata URL',
       'Metadata Date', 'Repository', 'Tags', 'Tags AAT URL',
       'Tags Wikidata URL', 'Artist', 'Medium_raw', 'Medium_classified',
       'Countr

save clean datasets in separate csv files

In [23]:
import pandas as pd

clean_data = []

# Define the desired columns
required_columns = ['Artist', 'Title', 'Medium', 'Medium_classified', 'Acquistion_classified', 
                    'Year_acquisition', 'Gender_classified', 'Artist_birth_year', 
                    'Artist_death_year', 'Country_calculated', 'Date_creation_year']

for df in transformed_data:
    # Add any missing columns as NaN
    for col in required_columns:
        if col not in df.columns:
            df[col] = pd.NA  # or use 'None' or 'NaN' depending on your preference

    clean_data.append(df[required_columns])



In [24]:
# Save each dataset using the corresponding name
for dataset, name in zip(transformed_data, dataset_names):
    # Generate a unique filename using the dataset name
    file_name = f"/Users/CUDAN/Documents/TLU/Data analysis/raw_data/{name}.csv"
    
    # Save the DataFrame to a CSV file
    dataset.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/met.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/reina_sofia.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/tate.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/pompidou.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/moma.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/whitney.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/national_gallery.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/kiasma.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/smk.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/ateneum.csv
Saved /Users/CUDAN/Documents/TLU/Data analysis/raw_data/queensland.csv


In [25]:

# Save each dataset using the corresponding name
for dataset, name in zip(clean_data, dataset_names):
    # Generate a unique filename using the dataset name
    file_name = f"/Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/{name}.csv"
    
    # Save the DataFrame to a CSV file
    dataset.to_csv(file_name, index=False)
    print(f"Saved {file_name}")

Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/met.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/reina_sofia.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/tate.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/pompidou.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/moma.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/whitney.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/national_gallery.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/kiasma.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/smk.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/ateneum.csv
Saved /Users/CUDAN/Documents/GitHub/data_analysis_portfolio/data/queensland.csv
